In [ ]:
# Import libraries and plotting functions
import pandas as pd
import numpy as np
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
# Import os and glob for file discovery
import os
import glob
from pathlib import Path
# Set default renderer for Plotly (optional)
pio.renderers.default = 'notebook_connected'

In [ ]:


# Define the DataSource folder path
data_source_path = Path('./DataSource')
# Initialize the dfs dictionary
dfs = {}
file_path_mapping = {}  # Store the mapping between key names and file paths

# Function to load all CSV files recursively from DataSource folder
def load_all_csv_files(base_path):
    """Load all CSV files from DataSource folder and subdirectories"""
    csv_files = []
    
    # Find all CSV files recursively
    for csv_file in base_path.rglob('*.csv'):
        csv_files.append(csv_file)
    
    return csv_files

# Load all CSV files
csv_files = load_all_csv_files(data_source_path)

print(f"Found {len(csv_files)} CSV files:")
for file_path in csv_files:
    # Create a meaningful key name from the file path
    relative_path = file_path.relative_to(data_source_path)
    # Create key by combining folder and filename (without extension)
    if len(relative_path.parts) > 1:
        key_name = f"{relative_path.parts[0]}_{relative_path.stem}"
    else:
        key_name = relative_path.stem
    
    try:
        # Load the CSV file
        df = pd.read_csv(file_path)
        dfs[key_name] = df
        file_path_mapping[key_name] = file_path  # Store the file path for later saving
        print(f"✅ Loaded: {relative_path} -> {key_name} ({len(df)} rows)")
    except Exception as e:
        print(f"❌ Failed to load {relative_path}: {str(e)}")

print(f"\nTotal DataFrames loaded: {len(dfs)}")
print("Available DataFrames:")
for key in sorted(dfs.keys()):
    print(f"  - {key}: {dfs[key].shape}")
    print(f"    Columns: {list(dfs[key].columns)}")

# Data cleaning for specific DataFrames (if they exist)
if 'GreenHouseGas_GreenHouseGasBySectors' in dfs:
    # focus on the 'MEASURE' column, discard '_SECTOR' 
    dfs['GreenHouseGas_GreenHouseGasBySectors']['MEASURE'] = dfs['GreenHouseGas_GreenHouseGasBySectors']['MEASURE'].str.replace('_SECTOR', '', regex=False)
    # focus on the 'Measure' column, replace 'Other sectors' with 'Other'
    if 'Measure' in dfs['GreenHouseGas_GreenHouseGasBySectors'].columns:
        dfs['GreenHouseGas_GreenHouseGasBySectors']['Measure'] = dfs['GreenHouseGas_GreenHouseGasBySectors']['Measure'].str.replace('Other sectors', 'Other', regex=False)

# Iterate through all dfs to clean each DataFrame
print("\nApplying data cleaning...")
for key, df in dfs.items():
    print(f"\nProcessing: {key}")
    original_rows = len(df)
    
    # Define the ideal columns we want to keep
    ideal_columns = ['REF_AREA', 'MEASURE', 'UNIT_MEASURE', 'TIME_PERIOD', 'OBS_VALUE', 'UNIT_MULT']
    
    # Find which columns actually exist in this DataFrame
    existing_columns = [col for col in ideal_columns if col in df.columns]
    print(f"  Available columns from ideal set: {existing_columns}")
    
    # Only select columns that actually exist
    if existing_columns:
        df = df[existing_columns]
        print(f"  Selected {len(existing_columns)} columns")
    else:
        print(f"  ⚠️  Warning: No ideal columns found, keeping all columns")
        # Keep all columns if none of the ideal ones exist
    
    # Remove EU regions if 'REF_AREA' column exists
    if 'REF_AREA' in df.columns:
        df_filtered = df[~df['REF_AREA'].isin(['EU27', 'EU', 'EU27_2020', 'EU28'])]
        removed_rows = original_rows - len(df_filtered)
        if removed_rows > 0:
            print(f"  Removed {removed_rows} EU region rows")
    else:
        df_filtered = df
        print(f"  No REF_AREA column found, skipping EU region removal")
    
    # Apply unit multiplier if both columns exist
    if 'UNIT_MULT' in df_filtered.columns and 'OBS_VALUE' in df_filtered.columns:
        df_filtered['OBS_VALUE'] = df_filtered['OBS_VALUE'] * (10 ** df_filtered['UNIT_MULT'])
        print(f"  Applied UNIT_MULT scaling to OBS_VALUE")
    else:
        missing_cols = [col for col in ['UNIT_MULT', 'OBS_VALUE'] if col not in df_filtered.columns]
        print(f"  Skipping UNIT_MULT scaling (missing columns: {missing_cols})")
    
    # Update the dataframe in the dictionary
    dfs[key] = df_filtered

print("\nData cleaning completed!")

# Save cleaned DataFrames back to their original files
print("\nSaving cleaned DataFrames back to CSV files...")
saved_count = 0
failed_count = 0

for key, df in dfs.items():
    if key in file_path_mapping:
        file_path = file_path_mapping[key]
        try:
            # Create a backup of the original file
            backup_path = file_path.with_suffix('.csv.backup')
            if file_path.exists() and not backup_path.exists():
                import shutil
                shutil.copy2(file_path, backup_path)
                print(f"📋 Created backup: {backup_path.name}")
            
            # Save the cleaned DataFrame
            df.to_csv(file_path, index=False)
            print(f"💾 Saved: {file_path.relative_to(data_source_path)} ({len(df)} rows)")
            saved_count += 1
            
        except Exception as e:
            print(f"❌ Failed to save {key}: {str(e)}")
            failed_count += 1
    else:
        print(f"⚠️  No file path found for {key}")
        failed_count += 1

print(f"\n✅ Successfully saved {saved_count} files")
if failed_count > 0:
    print(f"❌ Failed to save {failed_count} files")

print("\nFile overwriting completed!")
print("Note: Original files have been backed up with .backup extension")

Found 9 CSV files:
✅ Loaded: Energy\AgriculturalEnergyConsumption.csv -> Energy_AgriculturalEnergyConsumption (1760 rows)
✅ Loaded: GreenHouseGas\GreenHouseGasByNatureSources.csv -> GreenHouseGas_GreenHouseGasByNatureSources (19862 rows)
✅ Loaded: GreenHouseGas\GreenHouseGasBySectors.csv -> GreenHouseGas_GreenHouseGasBySectors (10531 rows)
✅ Loaded: GreenHouseGas\GreenHouseGasFromLULUCF.csv -> GreenHouseGas_GreenHouseGasFromLULUCF (4372 rows)
✅ Loaded: GreenHouseGas\GreenHouseGasWithLULUCF.csv -> GreenHouseGas_GreenHouseGasWithLULUCF (8694 rows)
✅ Loaded: GreenHouseGas\GreenHouseGasWithoutLULUCF.csv -> GreenHouseGas_GreenHouseGasWithoutLULUCF (8508 rows)
✅ Loaded: Land\AgriculturalLand.csv -> Land_AgriculturalLand (1649 rows)
✅ Loaded: Population\AnnualPopulationOECDCountry.csv -> Population_AnnualPopulationOECDCountry (1760 rows)
✅ Loaded: WaterAbstraction\AgriculturalWaterAbstraction.csv -> WaterAbstraction_AgriculturalWaterAbstraction (1865 rows)

Total DataFrames loaded: 9
Availabl